In [1]:
import sys, os
sys.path.append(os.getcwd() + '\..')
import pandas as pd
import numpy as np
from my_api import *
import matplotlib.pyplot as plt

0.9
12.685905874921438 8.685905874921438
0.5
15.15749916396364 11.15749916396364


d:\BaiduSyncdisk\Project\Co_simu_3\code_Cai2011\..\gen_S.py:139: ComplexWarning: Casting complex values to real discards the imaginary part
  S = S.astype(np.float64)


In [2]:
id = pd.MultiIndex.from_product([['fro', 2], [100, 300, 500]])
df = pd.DataFrame(index = id)

In [3]:
T = 300
rela = False
# cov_str = 'Cai2011Adaptive_Model2_my' # 数据有些问题，随机数种子
cov_str = 'Cai2011Adaptive_Model1'

In [4]:
simu_str = 'lx_band'
data_path = '../data_lxband_AR1_Grav_Cai/'
for N in [100, 300, 500]:
    if cov_str == 'Cai2011Adaptive_Model1':
        S = gen_S_Cai2011Adaptive_Model1(N)
    elif cov_str == 'Cai2011Adaptive_Model2_my':
        S = gen_S_Cai2011Adaptive_Model2_my(N = N, seed = 0)
    else:
        raise Exception
    
    for ord in ['fro', 2]:
        normS = np.linalg.norm(S, ord)
        
        for eta in [0.5, 0.8, 1]:
            cov_params = [cov_str]
            simu_params = [simu_str, eta]
            file_name = f"{ord},S,{N},{T},{cov_params},{simu_params}.txt"
            file_name = data_path + file_name
            data = np.loadtxt(file_name)

            col = str(simu_params)
            if col not in df.columns:
                df[col] = None
            if not rela:
                data *= normS # absolute error
            df[col][ord, N] = f"{data.mean() :.2f}({data.std() :.2f})"
df

['lx_band', 0.5] ['lx_band', 0.8] ['lx_band', 1]
fro 100       8.76(0.76)       6.54(0.31)     5.69(0.25)
    300      15.23(0.61)      10.61(0.44)     8.01(0.24)
    500      19.35(0.79)      13.24(0.58)     9.47(0.21)
2   100       3.48(0.38)       2.57(0.30)     2.53(0.23)
    300       5.00(0.41)       3.70(0.36)     3.18(0.27)
    500       5.39(0.38)       3.72(0.28)     3.24(0.25)

In [5]:
simu_str = '' # TBD
data_path = ''
num_cv = 50

for simu_str, data_path in [('LSRthreshold', '../data_Cai2011_LSRthreshold_cv50/'), ('thG', '../data_thG_4-25/')]:
    for N in [100, 300, 500]:
        if cov_str == 'Cai2011Adaptive_Model1':
            S = gen_S_Cai2011Adaptive_Model1(N)
        elif cov_str == 'Cai2011Adaptive_Model2_my':
            S = gen_S_Cai2011Adaptive_Model2_my(N = N, seed = 0)
        else:
            raise Exception
        
        for ord in ['fro', 2]:
            normS = np.linalg.norm(S, ord)
            
            for tau in [0.2]:
                # for prob in [0.99, 0.9, 1]:
                #     for qrob in [0, 0.01, 0.1]:
                for prob in [1, 0.99]:
                    for qrob in [0, 0.01]:
                        cov_params = [cov_str]
                        simu_params = [simu_str, tau, prob, qrob, 'brute', num_cv]
                        file_name = f"{ord},S,{N},{T},{cov_params},{simu_params}.txt"
                        file_name = data_path + file_name
                        
                        try:
                            data = np.loadtxt(file_name)
                        except FileNotFoundError:
                            continue

                        col = str(simu_params)
                        if col not in df.columns:
                            df[col] = None
                        if not rela:
                            data *= normS # absolute error
                        df[col][ord, N] = f"{data.mean() :.2f}({data.std() :.2f})"
df

['lx_band', 0.5] ['lx_band', 0.8] ['lx_band', 1]  \
fro 100       8.76(0.76)       6.54(0.31)     5.69(0.25)   
    300      15.23(0.61)      10.61(0.44)     8.01(0.24)   
    500      19.35(0.79)      13.24(0.58)     9.47(0.21)   
2   100       3.48(0.38)       2.57(0.30)     2.53(0.23)   
    300       5.00(0.41)       3.70(0.36)     3.18(0.27)   
    500       5.39(0.38)       3.72(0.28)     3.24(0.25)   

        ['LSRthreshold', 0.2, 1, 0, 'brute', 50]  \
fro 100                               8.44(0.60)   
    300                              20.42(3.34)   
    500                              31.70(4.61)   
2   100                               2.50(0.28)   
    300                               4.12(0.64)   
    500                               5.19(0.95)   

        ['thG', 0.2, 1, 0, 'brute', 50]  
fro 100                      8.59(0.54)  
    300                     22.58(1.08)  
    500                     34.78(2.92)  
2   100                      2.55(0.28)  
    300                      4.49(0.27)  
    500                      5.60(0.66)

In [6]:
simu_str = '' # to be determined
data_path1 = '../data_OtherMethods_4-5/'
data_path2 = '../data_Cai2011_other_threshold/'

for N in [100, 300, 500]:
    if cov_str == 'Cai2011Adaptive_Model1':
        S = gen_S_Cai2011Adaptive_Model1(N)
    elif cov_str == 'Cai2011Adaptive_Model2_my':
        S = gen_S_Cai2011Adaptive_Model2_my(N = N, seed = 0)
    else:
        raise Exception
    
    for ord in ['fro', 2]:
        normS = np.linalg.norm(S, ord)

        # from Cai2011_other import Other_Methods
        for simu_str in ['Sample', 'Soft Threshold', 'Hard Threshold', 'Linear Shrink', 'Nonlinear Shrink']: 
            data_path = data_path2 if 'Threshold' in simu_str else data_path1
            # print(data_path, simu_str)
 
            cov_params = [cov_str]
            simu_params = [simu_str]
            file_name = f"{ord},S,{N},{T},{cov_params},{simu_params}.txt"
            file_name = data_path + file_name
            try:
                data = np.loadtxt(file_name)
            except FileNotFoundError:
                continue

            col = str(*simu_params)
            if col not in df.columns:
                df[col] = None
            
            if not rela:
                data *= normS # absolute error
            df[col][ord, N] = f"{data.mean() :.2f}({data.std() :.2f})"
df

['lx_band', 0.5] ['lx_band', 0.8] ['lx_band', 1]  \
fro 100       8.76(0.76)       6.54(0.31)     5.69(0.25)   
    300      15.23(0.61)      10.61(0.44)     8.01(0.24)   
    500      19.35(0.79)      13.24(0.58)     9.47(0.21)   
2   100       3.48(0.38)       2.57(0.30)     2.53(0.23)   
    300       5.00(0.41)       3.70(0.36)     3.18(0.27)   
    500       5.39(0.38)       3.72(0.28)     3.24(0.25)   

        ['LSRthreshold', 0.2, 1, 0, 'brute', 50]  \
fro 100                               8.44(0.60)   
    300                              20.42(3.34)   
    500                              31.70(4.61)   
2   100                               2.50(0.28)   
    300                               4.12(0.64)   
    500                               5.19(0.95)   

        ['thG', 0.2, 1, 0, 'brute', 50]       Sample Soft Threshold  \
fro 100                      8.59(0.54)  14.57(0.33)     9.19(0.40)   
    300                     22.58(1.08)  43.55(0.37)    23.45(1.20)   
    500                     34.78(2.92)  72.36(0.43)    35.90(2.36)   
2   100                      2.55(0.28)   4.57(0.40)     2.85(0.35)   
    300                      4.49(0.27)   9.26(0.43)     4.31(0.27)   
    500                      5.60(0.66)  12.84(0.39)     5.56(0.51)   

        Hard Threshold Linear Shrink Nonlinear Shrink  
fro 100    13.81(0.39)   12.19(0.20)       7.49(0.30)  
    300    39.88(0.86)   29.02(0.11)             None  
    500    64.81(1.82)   41.34(0.09)             None  
2   100     4.12(0.35)    3.67(0.33)       3.50(0.39)  
    300     7.80(0.34)    5.59(0.16)             None  
    500    10.58(0.44)    6.27(0.11)             None

In [7]:
df

['lx_band', 0.5] ['lx_band', 0.8] ['lx_band', 1]  \
fro 100       8.76(0.76)       6.54(0.31)     5.69(0.25)   
    300      15.23(0.61)      10.61(0.44)     8.01(0.24)   
    500      19.35(0.79)      13.24(0.58)     9.47(0.21)   
2   100       3.48(0.38)       2.57(0.30)     2.53(0.23)   
    300       5.00(0.41)       3.70(0.36)     3.18(0.27)   
    500       5.39(0.38)       3.72(0.28)     3.24(0.25)   

        ['LSRthreshold', 0.2, 1, 0, 'brute', 50]  \
fro 100                               8.44(0.60)   
    300                              20.42(3.34)   
    500                              31.70(4.61)   
2   100                               2.50(0.28)   
    300                               4.12(0.64)   
    500                               5.19(0.95)   

        ['thG', 0.2, 1, 0, 'brute', 50]       Sample Soft Threshold  \
fro 100                      8.59(0.54)  14.57(0.33)     9.19(0.40)   
    300                     22.58(1.08)  43.55(0.37)    23.45(1.20)   
    500                     34.78(2.92)  72.36(0.43)    35.90(2.36)   
2   100                      2.55(0.28)   4.57(0.40)     2.85(0.35)   
    300                      4.49(0.27)   9.26(0.43)     4.31(0.27)   
    500                      5.60(0.66)  12.84(0.39)     5.56(0.51)   

        Hard Threshold Linear Shrink Nonlinear Shrink  
fro 100    13.81(0.39)   12.19(0.20)       7.49(0.30)  
    300    39.88(0.86)   29.02(0.11)             None  
    500    64.81(1.82)   41.34(0.09)             None  
2   100     4.12(0.35)    3.67(0.33)       3.50(0.39)  
    300     7.80(0.34)    5.59(0.16)             None  
    500    10.58(0.44)    6.27(0.11)             None

In [8]:
new_cols = []
for col in df.columns:
    try:
        col = eval(col)
    except Exception:
        pass

    if type(col) == str:
        new_col = col
    elif 'lx_band' in col:
        eta = col[1]
        new_col = fr'Augmented Band ($\eta={eta}$)'
    elif 'LSRthreshold' in col or 'thG' in col:
        formal_name = {'LSRthreshold': 'Augmented Threshold', 'thG': 'Threshold then Augment'}[col[0]]
        t, p, q = col[1:-2]
        new_col = fr'{formal_name} ($\tau={t}, p={p}, q={q}$)' # The r in fr, means raw string. Otherwise I should write \\tau instead of \tau.
    new_cols.append(new_col)
df.columns = new_cols
df.head(1)

,,Augmented Band ($\eta=0.5$),Augmented Band ($\eta=0.8$),Augmented Band ($\eta=1$),"Augmented Threshold ($\tau=0.2, p=1, q=0$)","Threshold then Augment ($\tau=0.2, p=1, q=0$)",Sample,Soft Threshold,Hard Threshold,Linear Shrink,Nonlinear Shrink
fro,100,8.76(0.76),6.54(0.31),5.69(0.25),8.44(0.60),8.59(0.54),14.57(0.33),9.19(0.40),13.81(0.39),12.19(0.20),7.49(0.30)


In [9]:
buf = '../latex/' + cov_str + '.tex'
caption = cov_str

with pd.option_context("max_colwidth", 1000, 'display.width', 50):
    df.to_latex(buf = buf, 
                index = True, na_rep = ' ', sparsify = True, 
             column_format = 'll|' + 'p{2cm}' * df.shape[1], 
             longtable = False, 
             escape = False, encoding = 'utf-8', 
             caption = caption, label = "my label", 
             position = 'htbp')
# escape参数，防止字符被转换使得我设的数学模式被破坏

C:\Users\lx\AppData\Local\Temp\ipykernel_18848\2522360471.py:5: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  df.to_latex(buf = buf,


In [10]:
pd.get_option('display.width')

80

In [11]:
lines = []

with open(buf, 'r') as fp:
    lines = fp.readlines()

s1 = r'\resizebox{\linewidth}{!}{' + '\n'
s2 = r'}' + '\n'
lines.insert(4, s1)
lines.insert(-1, s2)

with open(buf[:-4] + '_processed' + '.tex', 'w') as fp:
    fp.writelines(lines)
        # if not write_flag:
        #     if r'begin{tabular}' in line:
